In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=ada53f88dd50164402d9a4ccc3f812a9876d6281104e642c5c4d234a198fa965
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
#Import all libraries
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
#FOR SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter 
#FOR MLP
from sklearn.neural_network import MLPClassifier
#SVM
from sklearn.svm import SVC
#KNN
from sklearn.neighbors import KNeighborsClassifier
#Stacking
from vecstack import stacking
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [4]:
#Read training data file

trainfile = r'/gdrive/My Drive/CIS 508 Python/Assignment_3/RevisedHomesiteTrain.csv'
trainData = pd.read_csv(trainfile)

#Read test data file
testfile = r'/gdrive/My Drive/CIS 508 Python/Assignment_3/RevisedHomesiteTest.csv'
testData = pd.read_csv(testfile)

testData = testData.drop('GeographicField64', axis=1)
# copy of train data and test data to perform pre processing
cpy_traindata = trainData.copy()
cpy_testdata = testData.copy()

trainData.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0


In [0]:
#Shape and data types of data
print("Training Data shape:",trainData.shape)
print("Test Data shape:",testData.shape)
print("Training Data types:",trainData.dtypes)
# variable data types
trainData.describe()

Training Data shape: (65000, 596)
Test Data shape: (173836, 595)
Training Data types: CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
QuoteConversion_Flag    int64
Length: 596, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
count,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.00000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,...,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.0,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000,65000.000000
mean,7.798892,12.954892,9.014585,12.995831,9.060000,13.017308,9.064538,13.017877,8.973785,13.011031,13.814523,21.201046,11.465477,19.179246,953.073415,1.162747,18.262569,0.938339,0.001456,-1.0,25.0,7.370092,13.191985,10.711554,13.328754,7.781154,13.22280,-0.325600,13.964031,11.680662,13.316185,7.282015,13.181169,4.994108,12.984062,-0.333200,13.038754,11.867246,13.055738,7.318415,...,0.263846,0.063538,0.0,0.138262,0.111631,0.749954,0.000154,0.034877,0.934138,0.005231,0.025754,0.942938,0.057062,0.234092,0.038523,0.127277,0.600108,0.263923,0.735892,0.181385,0.036185,0.307969,0.474462,0.457569,0.542246,0.951569,0.048031,0.0,0.712754,0.287246,0.974969,0.020292,0.0,0.976262,0.023738,0.361862,0.120908,0.267631,0.249600,0.188646
std,5.304856,7.259023,5.586114,7.217093,5.568791,7.207118,5.588694,7.212825,5.564675,7.203325,5.105662,4.911451,6.893381,7.395318,290.710820,0.116634,7.405558,0.037123,0.001489,0.0,0.0,7.052165,7.157675,7.676760,7.201487,7.056926,7.11729,4.132777,6.076901,7.317099,7.222147,5.983242,7.121157,5.951460,7.191695,4.110041,7.141211,8.021028,7.219648,5.239252,...,0.440720,0.243931,0.0,0.345177,0.314914,0.433043,0.012403,0.183469,0.248042,0.072135,0.158401,0.231962,0.231962,0.423433,0.192457,0.333285,0.489880,0.440761,0.440860,0.385339,0.186751,0.461657,0.499351,0.498200,0.498216,0.214676,0.213833,0.0,0.452481,0.452481,0.156220,0.140999,0.0,0.152234,0.152234,0.480543,0.326022,0.442727,0.432785,0.391230
min,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,548.0

In [0]:
# check if same columns are present in Train and Test Data 
trainData.columns.difference(testData.columns)
# All should be same except target column

Index(['QuoteConversion_Flag'], dtype='object')

In [0]:
# check if there are any missing values in any column
print("Number of null values in train data",trainData.isnull().sum().sum())
print("Number of null values in test data",testData.isnull().sum().sum())
# no missing values

Number of null values in train data 0
Number of null values in test data 0


In [0]:
# Percentage split of target variable
per_split = pd.DataFrame(trainData.groupby('QuoteConversion_Flag')['QuoteConversion_Flag'].count())
per_split['QuoteConversion_Flag'] = 100*per_split['QuoteConversion_Flag']/trainData.shape[0]
per_split
# unbalanced dataset

,QuoteConversion_Flag
QuoteConversion_Flag,
0,81.135385
1,18.864615


In [5]:
# Remove ID column from training and test dataset
cpy_traindata.drop("QuoteNumber", axis=1, inplace=True)
cpy_testdata.drop("QuoteNumber", axis=1, inplace=True)
print("After removing QuoteNumber column,training data shape: ",cpy_traindata.shape)
print("After removing QuoteNumber column test data shape: ",cpy_testdata.shape)

After removing QuoteNumber column,training data shape:  (65000, 595)
After removing QuoteNumber column test data shape:  (173836, 594)


In [6]:
# Remove columns from training and test dataset which have 0 standard deviation 
counter_col_removed = 0
for (columnName, columnData) in cpy_traindata.iteritems():
    std_dev = ( round(cpy_traindata[columnName].std(),2))
    if (std_dev == 0):
        cpy_traindata = cpy_traindata.drop(columns = columnName)
        cpy_testdata = cpy_testdata.drop(columns = columnName)
        print(columnName)
        counter_col_removed = counter_col_removed + 1
        
print("After removing columns:",counter_col_removed, "training data shape: ",cpy_traindata.shape)
print("After removing columns:",counter_col_removed, "test data shape: ",cpy_testdata.shape)

Field9
GeographicField10A
GeographicField10B
PropertyField6
PersonalField16_XG
PersonalField16_XT
PersonalField16_XV
PersonalField16_XY
PersonalField16_YG
PersonalField16_ZI
PersonalField16_ZM
PersonalField16_ZO
PersonalField16_ZP
PersonalField16_ZS
PersonalField17_XD
PersonalField17_XF
PersonalField17_XM
PersonalField17_XZ
PersonalField17_YG
PersonalField17_YO
PersonalField17_YP
PersonalField17_ZB
PersonalField17_ZD
PersonalField17_ZJ
PersonalField18_XB
PersonalField18_XX
PersonalField18_ZB
PersonalField19_XH
PersonalField19_XS
PersonalField19_ZS
PropertyField7_B
PropertyField7_T
PropertyField37_ 
GeographicField63_ 
After removing columns: 34 training data shape:  (65000, 561)
After removing columns: 34 test data shape:  (173836, 560)


In [7]:
#Select just Target Column from training dataset
Y = cpy_traindata.iloc[:, -1]

#Select features from training and test dataset
X = cpy_traindata.iloc[:, :-1]
X_Test_A = cpy_testdata

#split the entire training data in 
X_Train, X_Test_1, Y_Train, Y_Test_1 = train_test_split( X, Y, test_size=0.20, random_state=42)
print("Initial shape for entire data:",cpy_traindata.shape)
print("Shape of new training data:", X_Train.shape)
print("Shape of new test split data:", X_Test_1.shape)

Initial shape for entire data: (65000, 561)
Shape of new training data: (52000, 560)
Shape of new test split data: (13000, 560)


In [0]:
 #check percentage of data after splitting
 m = Y_Train.groupby(Y_Train.values).count()
 m

0    42183
1     9817
Name: QuoteConversion_Flag, dtype: int64

# SMOTE

In [8]:
# Resample the minority class. 
sm = SMOTE(sampling_strategy='minority', random_state=7 )

# Fit the model to generate the data.
oversampled_trainX, oversampled_trainY = sm.fit_sample(X_Train, Y_Train)
print('Resampled dataset shape %s' % Counter(oversampled_trainY))

Resampled dataset shape Counter({0: 42183, 1: 42183})


In [0]:
X_Train = oversampled_trainX 
Y_Train = oversampled_trainY

# MLP - Neural network

In [0]:
# neural networks - mlp
nn_clf = MLPClassifier()
nn_clf.fit(X_Train, Y_Train) 

#Prediction on training data
pred=pd.DataFrame(nn_clf.predict(X_Train),columns=["Prediction"])
print("----------------------------Neural Networks - Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Neural networks:",accuracy_score(Y_Train,pred["Prediction"]))
print("Confusion Matrix on training data using Neural networks\n", confusion_matrix(Y_Train,pred["Prediction"]))

#Prediction on test data created from split
pred=pd.DataFrame(nn_clf.predict(X_Test_1),columns=["Prediction"])
print("----------------------------Neural Networks - Test Data created from split-------------------------------------\n")
print("Accuracy Score on training data using Neural networks:",accuracy_score(Y_Test_1,pred["Prediction"]))
print("Confusion Matrix on training data using Neural networks\n", confusion_matrix(Y_Test_1,pred["Prediction"]))
print("Classification report\n", classification_report(Y_Test_1,pred["Prediction"]))

----------------------------Neural Networks - Training Data------------------------------------------------

Accuracy Score on training data using Neural networks: 0.7390892065524026
Confusion Matrix on training data using Neural networks
 [[41885   298]
 [21714 20469]]
----------------------------Neural Networks - Test Data created from split-------------------------------------

Accuracy Score on training data using Neural networks: 0.8926923076923077
Confusion Matrix on training data using Neural networks
 [[10451   104]
 [ 1291  1154]]
Classification report
               precision    recall  f1-score   support

           0       0.89      0.99      0.94     10555
           1       0.92      0.47      0.62      2445

    accuracy                           0.89     13000
   macro avg       0.90      0.73      0.78     13000
weighted avg       0.90      0.89      0.88     13000



In [0]:
# get default parameters for multi layer perceptron
nn_clf.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [0]:
#Random Hyperparameter tuning for neural networks model
print("RandomizedSearchCV-Multi Layer Perceptron")
parameters={'activation': ['logistic','relu'], 'hidden_layer_sizes' : range(100,130,10), 'solver' : ['sgd', 'adam']}
nn_random = RandomizedSearchCV(nn_clf,parameters,n_iter=10, cv=8, scoring ="precision")
nn_random.fit(X_Train, Y_Train)
nn_rand_parm=nn_random.best_params_
print(nn_rand_parm)

RandomizedSearchCV-Multi Layer Perceptron
{'solver': 'adam', 'hidden_layer_sizes': 100, 'activation': 'relu'}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the MLP
nn_clf_r = MLPClassifier(**nn_rand_parm)
#grid model
nn_clf_r.fit(X_Train,Y_Train)

print("----------------------------Hypertuning using Random Search MLP------------------------------------------------\n")
#random model Prediction on training data
pred_nn=pd.DataFrame(nn_clf_r.predict(X_Train),columns=["Prediction"])
print("Accuracy Score on training data using MLP - random Search:",accuracy_score(Y_Train,pred_nn["Prediction"]))
print("Confusion Matrix on training data using MLP - random Search\n", confusion_matrix(Y_Train,pred_nn["Prediction"]))

# random model prediction on test data
pred_nn=pd.DataFrame(nn_clf_r.predict(X_Test_1),columns=["Prediction"])
print("Accuracy Score on test data using MLP - random Search:",accuracy_score(Y_Test_1,pred_nn["Prediction"]))
print("Balanced Accuracy Score on test data using MLP - random Search:",balanced_accuracy_score(Y_Test_1,pred_nn["Prediction"]))
print("Confusion Matrix on test data using MLP - random Search\n", confusion_matrix(Y_Test_1,pred_nn["Prediction"]))
print("Classification report on test data using MLP - random Search\n", classification_report(Y_Test_1,pred_nn["Prediction"]))

----------------------------Hypertuning using Random Search MLP------------------------------------------------

Accuracy Score on training data using MLP - random Search: 0.8720574639072612
Confusion Matrix on training data using MLP - random Search
 [[34271  7912]
 [ 2882 39301]]
Accuracy Score on test data using MLP - random Search: 0.8196923076923077
Balanced Accuracy Score on test data using MLP - random Search: 0.8375817002961409
Confusion Matrix on test data using MLP - random Search
 [[8538 2017]
 [ 327 2118]]
Classification report on test data using MLP - random Search
               precision    recall  f1-score   support

           0       0.96      0.81      0.88     10555
           1       0.51      0.87      0.64      2445

    accuracy                           0.82     13000
   macro avg       0.74      0.84      0.76     13000
weighted avg       0.88      0.82      0.84     13000



In [0]:
#mlp on actual test data using default model
pred_test_nn=pd.DataFrame(nn_clf.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_nn = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_test_nn["Prediction"]})
submission_nn.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_mlp.csv", index = None)

In [0]:
#mlp on actual test data using tuned model
pred_test_nn=pd.DataFrame(nn_clf_r.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_nn = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_test_nn["Prediction"]})
submission_nn.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_mlp_1.csv", index = None)

# SVM

In [11]:
#svm classifier linear svc
from sklearn.svm import LinearSVC
svm_clf = LinearSVC()
svm_clf.fit(X_Train, Y_Train) 

#Prediction on training data
pred_svm=pd.DataFrame(svm_clf.predict(X_Train),columns=["Prediction"])
print("----------------------------SVC - Training Data------------------------------------------------\n")
print("Accuracy Score on training data:",accuracy_score(Y_Train,pred_svm["Prediction"]))
print("Confusion Matrix on training data\n", confusion_matrix(Y_Train,pred_svm["Prediction"]))

#Prediction on test data created from split
pred_svm=pd.DataFrame(svm_clf.predict(X_Test_1),columns=["Prediction"])
print("----------------------------SVC - Test Data created from split-------------------------------------\n")
print("Accuracy Score on test data:",accuracy_score(Y_Test_1,pred_svm["Prediction"]))
print("Confusion Matrix on test data\n", confusion_matrix(Y_Test_1,pred_svm["Prediction"]))
print("Classification report\n", classification_report(Y_Test_1,pred_svm["Prediction"]))

----------------------------SVM - Training Data------------------------------------------------

Accuracy Score on training data: 0.8051730769230769
Confusion Matrix on training data
 [[36797  5386]
 [ 4745  5072]]
----------------------------SVM - Test Data created from split-------------------------------------

Accuracy Score on training data: 0.81
Confusion Matrix on training data
 [[9246 1309]
 [1161 1284]]
Classification report
               precision    recall  f1-score   support

           0       0.89      0.88      0.88     10555
           1       0.50      0.53      0.51      2445

    accuracy                           0.81     13000
   macro avg       0.69      0.70      0.70     13000
weighted avg       0.81      0.81      0.81     13000



In [13]:
#Random Hyperparameter tuning for svm
print("RandomizedSearchCV- SVC")
parameters={'loss': ['hinge', 'squared_hinge'], 'C' : range(1,10,2)}
svm_random = RandomizedSearchCV(svm_clf,parameters,n_iter=10, cv=3, scoring ="precision")
svm_random.fit(X_Train, Y_Train)
svm_rand_parm=svm_random.best_params_
print(svm_rand_parm)

RandomizedSearchCV- SVC
{'loss': 'squared_hinge', 'C': 7}


In [14]:
#Using the parameters obtained from HyperParameterTuning in the SVM
svm_clf_r = LinearSVC(**svm_rand_parm)
#grid model
svm_clf_r.fit(X_Train,Y_Train)

print("----------------------------Hypertuning using Random Search SVM------------------------------------------------\n")
#random model Prediction on training data
pred_svm=pd.DataFrame(svm_clf_r.predict(X_Train),columns=["Prediction"])
print("Accuracy Score on training data using SVM - random Search:",accuracy_score(Y_Train,pred_svm["Prediction"]))
print("Confusion Matrix on training data using SVM - random Search\n", confusion_matrix(Y_Train,pred_svm["Prediction"]))

# random model prediction on test data
pred_svm=pd.DataFrame(svm_clf_r.predict(X_Test_1),columns=["Prediction"])
print("Accuracy Score on test data using SVM - random Search:",accuracy_score(Y_Test_1,pred_svm["Prediction"]))
print("Balanced Accuracy Score on test data using SVM - random Search:",balanced_accuracy_score(Y_Test_1,pred_svm["Prediction"]))
print("Confusion Matrix on test data using SVM - random Search\n", confusion_matrix(Y_Test_1,pred_svm["Prediction"]))
print("Classification report on test data using SVM - random Search\n", classification_report(Y_Test_1,pred_svm["Prediction"]))

----------------------------Hypertuning using Random Search SVM------------------------------------------------

Accuracy Score on training data using SVM - random Search: 0.3948076923076923
Confusion Matrix on training data using SVM - random Search
 [[11865 30318]
 [ 1152  8665]]
Accuracy Score on test data using SVM - random Search: 0.39061538461538464
Balanced Accuracy Score on test data using SVM - random Search: 0.5766464298895938
Confusion Matrix on test data using SVM - random Search
 [[2939 7616]
 [ 306 2139]]
Classification report on test data using SVM - random Search
               precision    recall  f1-score   support

           0       0.91      0.28      0.43     10555
           1       0.22      0.87      0.35      2445

    accuracy                           0.39     13000
   macro avg       0.56      0.58      0.39     13000
weighted avg       0.78      0.39      0.41     13000



In [0]:
#svm on actual test data using default model
pred_test_svm=pd.DataFrame(svm_clf.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_svm = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_test_svm["Prediction"]})
submission_svm.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_SVC_Linear_def.csv", index = None)

# KNN

In [0]:
#KNN classifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_Train, Y_Train) 

#Prediction on training data
pred_knn=pd.DataFrame(knn_clf.predict(X_Train),columns=["Prediction"])
print("----------------------------KNN - Training Data------------------------------------------------\n")
print("Accuracy Score on training data using KNN:",accuracy_score(Y_Train,pred_knn["Prediction"]))
print("Confusion Matrix on training data using KNN\n", confusion_matrix(Y_Train,pred_knn["Prediction"]))

#Prediction on test data created from split
pred_knn=pd.DataFrame(knn_clf.predict(X_Test_1),columns=["Prediction"])
print("----------------------------KNN - Test Data created from split-------------------------------------\n")
print("Accuracy Score on training data using KNN:",accuracy_score(Y_Test_1,pred_knn["Prediction"]))
print("Confusion Matrix on training data using KNN\n", confusion_matrix(Y_Test_1,pred_knn["Prediction"]))
print("Classification report\n", classification_report(Y_Test_1,pred_knn["Prediction"]))

----------------------------KNN - Training Data------------------------------------------------

Accuracy Score on training data using KNN: 0.7981651376146789
Confusion Matrix on training data using KNN
 [[25592 16591]
 [  437 41746]]
----------------------------KNN - Test Data created from split-------------------------------------

Accuracy Score on training data using KNN: 0.5234615384615384
Confusion Matrix on training data using KNN
 [[5191 5364]
 [ 831 1614]]
Classification report
               precision    recall  f1-score   support

           0       0.86      0.49      0.63     10555
           1       0.23      0.66      0.34      2445

    accuracy                           0.52     13000
   macro avg       0.55      0.58      0.48     13000
weighted avg       0.74      0.52      0.57     13000



In [0]:
# get default parameters for multi layer perceptron
knn_clf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [0]:
#Random Hyperparameter tuning for KNN
print("RandomizedSearchCV- KNN")
parameters={'algorithm': ['auto', 'kd_tree'], 'n_neighbors' : range(5,10,1), 'p': range(2,5,1)}
knn_random = RandomizedSearchCV(knn_clf,parameters,n_iter=10, cv=3, scoring ="precision")
knn_random.fit(X_Train, Y_Train)
knn_rand_parm=knn_random.best_params_
print(knn_rand_parm)

RandomizedSearchCV- KNN
{'p': 4, 'n_neighbors': 6, 'algorithm': 'auto'}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the KNN
knn_clf_r = KNeighborsClassifier(**knn_rand_parm)
#grid model
knn_clf_r.fit(X_Train,Y_Train)

print("----------------------------Hypertuning using Random Search KNN------------------------------------------------\n")
#random model Prediction on training data
pred_knn=pd.DataFrame(knn_clf_r.predict(X_Train),columns=["Prediction"])
print("Accuracy Score on training data using SVM - random Search:",accuracy_score(Y_Train,pred_knn["Prediction"]))
print("Confusion Matrix on training data using SVM - random Search\n", confusion_matrix(Y_Train,pred_knn["Prediction"]))

# random model prediction on test data
pred_knn=pd.DataFrame(knn_clf_r.predict(X_Test_1),columns=["Prediction"])
print("Accuracy Score on test data using SVM - random Search:",accuracy_score(Y_Test_1,pred_knn["Prediction"]))
print("Balanced Accuracy Score on test data using SVM - random Search:",balanced_accuracy_score(Y_Test_1,pred_knn["Prediction"]))
print("Confusion Matrix on test data using SVM - random Search\n", confusion_matrix(Y_Test_1,pred_knn["Prediction"]))
print("Classification report on test data using SVM - random Search\n", classification_report(Y_Test_1,pred_knn["Prediction"]))

----------------------------Hypertuning using Random Search SVM------------------------------------------------

Accuracy Score on training data using SVM - random Search: 0.7935068629542706
Confusion Matrix on training data using SVM - random Search
 [[27301 14882]
 [ 2539 39644]]
Accuracy Score on test data using SVM - random Search: 0.5593846153846154
Balanced Accuracy Score on test data using SVM - random Search: 0.5757737975876677
Confusion Matrix on test data using SVM - random Search
 [[5800 4755]
 [ 973 1472]]
Classification report on test data using SVM - random Search
               precision    recall  f1-score   support

           0       0.86      0.55      0.67     10555
           1       0.24      0.60      0.34      2445

    accuracy                           0.56     13000
   macro avg       0.55      0.58      0.50     13000
weighted avg       0.74      0.56      0.61     13000



In [0]:
#KNN on actual test data
pred_test_knn=pd.DataFrame(knn_clf_r.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_knn = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_test_knn["Prediction"]})
submission_knn.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_KNN.csv", index = None)

# Decision Tree

In [0]:
#Make default decision tree using training dataset
clf = DecisionTreeClassifier()
clf.fit(X_Train, Y_Train)

#Prediction on training data
pred=pd.DataFrame(clf.predict(X_Train),columns=["Prediction"])
print("----------------------------Decision Tree - Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Decision Tree:",accuracy_score(Y_Train,pred["Prediction"]))
print("Confusion Matrix on training data using Decision Tree\n", confusion_matrix(Y_Train,pred["Prediction"]))

#Prediction on test data created from split
pred=pd.DataFrame(clf.predict(X_Test_1),columns=["Prediction"])
print("----------------------------Decision Tree - Test Data created from split-------------------------------------\n")
print("Accuracy Score on test data:",accuracy_score(Y_Test_1,pred["Prediction"]))
print("Confusion Matrix on test data\n", confusion_matrix(Y_Test_1,pred["Prediction"]))
print("Classification report\n", classification_report(Y_Test_1,pred["Prediction"]))

----------------------------Decision Tree - Training Data------------------------------------------------

Accuracy Score on training data using Decision Tree: 1.0
Confusion Matrix on training data using Decision Tree
 [[42183     0]
 [    0 42183]]
----------------------------Decision Tree - Test Data created from split-------------------------------------

Accuracy Score on training data: 0.882
Confusion Matrix on training data
 [[9712  843]
 [ 691 1754]]
Classification report
               precision    recall  f1-score   support

           0       0.93      0.92      0.93     10555
           1       0.68      0.72      0.70      2445

    accuracy                           0.88     13000
   macro avg       0.80      0.82      0.81     13000
weighted avg       0.89      0.88      0.88     13000



In [0]:
print("RandomizedSearchCV-Decision tree")
parameters={'max_depth': range(10,100,10), 'min_samples_leaf' : range(2,25,5), 'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=20, cv=5, scoring ="precision")
clf_random.fit(X_Train, Y_Train)
rand_parm=clf_random.best_params_
print(rand_parm)

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 22, 'max_depth': 20, 'criterion': 'gini'}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf_r = DecisionTreeClassifier(**rand_parm)
clf_r.fit(X_Train,Y_Train)

print("\n--------------------------Hypertuning using Random Search------------------------------------------------\n")
#Random model Prediction on training data
pred_r=pd.DataFrame(clf_r.predict(X_Train),columns=["Prediction"])
print("Accuracy Score on training data using Decision Tree - Random Search:",accuracy_score(Y_Train,pred_r["Prediction"]))
print("Confusion Matrix on training data using Decision Tree - Random Search\n", confusion_matrix(Y_Train,pred_r["Prediction"]))

# Random model prediction on test data
pred_r=pd.DataFrame(clf_r.predict(X_Test_1),columns=["Prediction"])
print("Accuracy Score on test data using Decision Tree - Random Search:",accuracy_score(Y_Test_1,pred_r["Prediction"]))
print("Balanced Accuracy Score on test data using Decision Tree - Random Search:",balanced_accuracy_score(Y_Test_1,pred_r["Prediction"]))
print("Confusion Matrix on test data using Decision Tree - Random Search\n", confusion_matrix(Y_Test_1,pred_r["Prediction"]))
print("Classification report on test data using Decision Tree - Random Search\n", classification_report(Y_Test_1,pred_r["Prediction"]))


--------------------------Hypertuning using Random Search------------------------------------------------

Accuracy Score on training data using Decision Tree - Random Search: 0.9550766896617121
Confusion Matrix on training data using Decision Tree - Random Search
 [[41015  1168]
 [ 2622 39561]]
Accuracy Score on test data using Decision Tree - Random Search: 0.9048461538461539
Balanced Accuracy Score on test data using Decision Tree - Random Search: 0.8238703877536984
Confusion Matrix on test data using Decision Tree - Random Search
 [[10066   489]
 [  748  1697]]
Classification report on test data using Decision Tree - Random Search
               precision    recall  f1-score   support

           0       0.93      0.95      0.94     10555
           1       0.78      0.69      0.73      2445

    accuracy                           0.90     13000
   macro avg       0.85      0.82      0.84     13000
weighted avg       0.90      0.90      0.90     13000



In [0]:
#Decision tree on actual test data
pred_dt=pd.DataFrame(clf_r.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_dt = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_dt["Prediction"]})
submission_dt.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_DT.csv", index = None)

# Random Forest

In [0]:
#Random forest
rfc = RandomForestClassifier()
rfc.fit(X_Train, Y_Train)

#Prediction on training data
pred_rf=pd.DataFrame(rfc.predict(X_Train),columns=["Prediction"])
print("----------------------------Random Forest: Training Data------------------------------------------------\n")
print("Accuracy Score on training data using Random Forest:",accuracy_score(Y_Train,pred_rf["Prediction"]))
print("Confusion Matrix on training data using Random Forest\n", confusion_matrix(Y_Train,pred_rf["Prediction"]))

# prediction on test data
pred_rf=pd.DataFrame(rfc.predict(X_Test_1),columns=["Prediction"])
print("----------------------------Random Forest: Test Data------------------------------------------------\n")
print("Accuracy Score on test data using Random Forest:",accuracy_score(Y_Test_1,pred_rf["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest:",balanced_accuracy_score(Y_Test_1,pred_rf["Prediction"]))
print("Confusion Matrix on test data using Random Forest\n", confusion_matrix(Y_Test_1,pred_rf["Prediction"]))
print("Classification report on test data using Random Forest\n", classification_report(Y_Test_1,pred_rf["Prediction"]))


----------------------------Random Forest: Training Data------------------------------------------------

Accuracy Score on training data using Random Forest: 0.9961477372401204
Confusion Matrix on training data using Random Forest
 [[42161    22]
 [  303 41880]]
----------------------------Random Forest: Test Data------------------------------------------------

Accuracy Score on test data using Random Forest: 0.8943846153846153
Balanced Accuracy Score on test data using Random Forest: 0.769818140250843
Confusion Matrix on test data using Random Forest
 [[10233   322]
 [ 1051  1394]]
Classification report on test data using Random Forest
               precision    recall  f1-score   support

           0       0.91      0.97      0.94     10555
           1       0.81      0.57      0.67      2445

    accuracy                           0.89     13000
   macro avg       0.86      0.77      0.80     13000
weighted avg       0.89      0.89      0.89     13000



In [0]:
#Hyperparameter tuning done for random forest classifier
#RANDOM SEARCH--------------------------------------------

print("RandomizedSearchCV-Decision tree")
parameters={'max_features': range(10,50,5) ,'n_estimators':[25,30,40,50], 'criterion':['gini','entropy']}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=20,cv=5, scoring ='precision')
rfc_random.fit(X_Train, Y_Train)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

RandomizedSearchCV-Decision tree
{'n_estimators': 50, 'max_features': 45, 'criterion': 'entropy'}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the randomforestclassifier
rfc_r = RandomForestClassifier(**rand_parm_rfc)
rfc_r.fit(X_Train,Y_Train)
rfc_r_predict = rfc_r.predict(X_Test_1)

print("\n--------------------------Hypertuning using Random Search------------------------------------------------\n")
#Random model Prediction on training data
pred_r_rfc=pd.DataFrame(rfc_r.predict(X_Train),columns=["Prediction"])
print("Accuracy Score on training data using Random Forest - Random Search:",accuracy_score(Y_Train,pred_r_rfc["Prediction"]))
print("Confusion Matrix on training data using Random Forest - Random Search\n", confusion_matrix(Y_Train,pred_r_rfc["Prediction"]))


# Random model prediction on test data
pred_r_rfc=pd.DataFrame(rfc_r.predict(X_Test_1),columns=["Prediction"])
print("Accuracy Score on test data using Random Forest - Random Search:",accuracy_score(Y_Test_1,pred_r_rfc["Prediction"]))
print("Balanced Accuracy Score on test data using Random Forest - Random Search:",balanced_accuracy_score(Y_Test_1,pred_r_rfc["Prediction"]))
print("Confusion Matrix on test data using Random Forest - Random Search\n", confusion_matrix(Y_Test_1,pred_r_rfc["Prediction"]))
print("Classification report on test data using Random Forest - Random Search\n", classification_report(Y_Test_1,pred_r_rfc["Prediction"]))



--------------------------Hypertuning using Random Search------------------------------------------------

Accuracy Score on training data using Random Forest - Random Search: 0.9999288813028945
Confusion Matrix on training data using Random Forest - Random Search
 [[42183     0]
 [    6 42177]]
Accuracy Score on test data using Random Forest - Random Search: 0.919
Balanced Accuracy Score on test data using Random Forest - Random Search: 0.8198592628543253
Confusion Matrix on test data using Random Forest - Random Search
 [[10331   224]
 [  829  1616]]
Classification report on test data using Random Forest - Random Search
               precision    recall  f1-score   support

           0       0.93      0.98      0.95     10555
           1       0.88      0.66      0.75      2445

    accuracy                           0.92     13000
   macro avg       0.90      0.82      0.85     13000
weighted avg       0.92      0.92      0.91     13000



In [0]:
#Random forest on actual test data
pred_rf=pd.DataFrame(rfc_r.predict(X_Test_A),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_rf = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_rf["Prediction"]})
submission_rf.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_RF.csv", index = None)

# Stacking

In [0]:
#STACKING MODELS =====================================================================
print("________\nEnsemble Methods Predictions using MLP, RandomForest and Decision Tree Classifier\n")

models = [ MLPClassifier(), RandomForestClassifier(**rand_parm_rfc), 
          DecisionTreeClassifier(**rand_parm) ]
      
S_Train, S_Test = stacking(models,                   
                           X_Train, Y_Train, X_Test_A,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=3, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPClassifier]
    fold  0:  [0.86348766]
    fold  1:  [0.74663964]
    fold  2:  [0.63896593]
    ----
    MEAN:     [0.74969775] + [0.09168611]
    FULL:     [0.74969775]

model  1:     [RandomForestClassifier]
    fold  0:  [0.94843894]
    fold  1:  [0.94662542]
    fold  2:  [0.94509637]
    ----
    MEAN:     [0.94672024] + [0.00136625]
    FULL:     [0.94672024]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.93311287]
    fold  1:  [0.93396629]
    fold  2:  [0.93321954]
    ----
    MEAN:     [0.93343290] + [0.00037967]
    FULL:     [0.93343290]



In [0]:
#STACKING - CONTRUCT A Random forest model==============================
model_st = RandomForestClassifier()
    
model_st = model_st.fit(S_Train, Y_Train)

pred_st=pd.DataFrame(model_st.predict(S_Test),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_st = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_st["Prediction"]})
submission_st.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_ST.csv", index = None)

In [0]:
S_Train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [0]:
#Hyperparameter tuning done for stacking random forest classifier
#RANDOM SEARCH--------------------------------------------

print("RandomizedSearchCV-Stacking")
parameters={'max_features': range(1,3,1) ,'n_estimators':[25,30,40,50], 'criterion':['gini','entropy']}
st_random = RandomizedSearchCV(model_st,parameters,n_iter=20,cv=5, scoring ='precision')
st_random.fit(S_Train, Y_Train)
rand_parm_st=st_random.best_params_
print(rand_parm_st)

RandomizedSearchCV-Stacking
{'n_estimators': 25, 'max_features': 1, 'criterion': 'gini'}


In [0]:
#STACKING - CONTRUCT A Random forest model==============================
model_st = RandomForestClassifier(**rand_parm_st)
    
model_st = model_st.fit(S_Train, Y_Train)

pred_st=pd.DataFrame(model_st.predict(S_Test),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_st = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_st["Prediction"]})
submission_st.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_ST_tuned.csv", index = None)

In [0]:
#STACKING - CONTRUCT A MLP model==============================
model_st_mlp = MLPClassifier()
    
model_st_mlp = model_st_mlp.fit(S_Train, Y_Train)

pred_st_mlp=pd.DataFrame(model_st_mlp.predict(S_Test),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_st_mlp = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_st_mlp["Prediction"]})
submission_st_mlp.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_ST_MLP.csv", index = None)

In [0]:
#STACKING - CONTRUCT A Decision model==============================
model_st_dt = DecisionTreeClassifier(**rand_parm)
    
model_st_dt = model_st_dt.fit(S_Train, Y_Train)

pred_st_dt=pd.DataFrame(model_st_dt.predict(S_Test),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_st_dt = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_st_dt["Prediction"]})
submission_st_dt.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_ST_DT.csv", index = None)

In [0]:
#STACKING - CONTRUCT A Decision model==============================
model_st_GB = GradientBoostingClassifier()
    
model_st_GB = model_st_GB.fit(S_Train, Y_Train)

pred_st_GB=pd.DataFrame(model_st_GB.predict(S_Test),columns=["Prediction"])

QuoteNumber = testData['QuoteNumber']
submission_st_GB = pd.DataFrame({"QuoteNumber": QuoteNumber, "QuoteConversion_Flag":pred_st_GB["Prediction"]})
submission_st_GB.to_csv("/gdrive/My Drive/CIS 508 Python/Assignment_3/Submission_ST_GB.csv", index = None)